# Repeat-Until-Success Sample
This sample shows the use of branching on measurement and qubit reuse to implement a Repeat-Until-Success algorithm on Quantinuum hardware targets. It is drawn from a collaboration between Microsoft and Quantinuum research teams, specifically the authors Natalie C. Brown, John Peter Campora III, Ciarán Ryan-Anderson, Dominic Lucchetti, and Alex Chernoguzov from Quantinuum; Stefan Wernli, Adam Paetznick, Martin Roetteler, and Krysta Svore from Microsft; as well as Bettina Heim and Cassandra Granade.

The program uses RUS and two sub-circuits to perform the single-qubit unitary V<sub>3</sub> = (I + 2iZ)/(√5) as described in detail in [Repeat-Until-Success: Non-deterministic decomposition of single-qubit unitaries](https://arxiv.org/abs/1311.1074) by Adam Paetznik and Krysta Svore.



## 1. Connect to Azure Quantum and set up target

First, we must configure the qsharp module to connect the azure workspace and specify a target. The Quantinuum syntax checker target `quantinuum.sim.h2-1sc` is chosen by default, which will perform program compilation but always return all zeros. To update to target that performs hardware-modeled noisy simulation, comment out the first target and uncomment the selection of `quantinuum.sim.h2-1e`. Not that this will consume quota as part of executing the simulation.

We also configure the target capability as `AdaptiveExecution` to indicate we use the QIR profile with support for mid-circuit measurement, measurement-based control flow, and classical value support as part of compilation.

Replace the `resourceId` and `location` connection parameters with the values for your configured Azure Quantum Workspace

In [ ]:
import time
import qsharp
import qsharp.azure

# Set the compiler target to the one queried from the workspace
targets = qsharp.azure.connect(
   resourceId="",
   location=""
)


qsharp.azure.target("quantinuum.sim.h2-1sc") # Syntax-Checker that verifies connectivity and compilation
                                             # Always returns all zero results to indicate success
# qsharp.azure.target("quantinuum.sim.h2-1e") # Emulator target that will run the program with hardware-modeled noise
                                              # Uses HQC/quota
qsharp.azure.target_capability("AdaptiveExecution")


## 2. Compile the RUS Program
<i>Note: compiling this code will produce expected warnings about potentially incompatible capabilities from the utility operation `PreparePauliEigenstate`. The incompatible code paths in the library are not used by this sample and teh warning can be safely ignored.</i>

In [ ]:
%%qsharp
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Diagnostics;

/// # Summary
/// Example of a Repeat-until-success algorithm implementing a circuit
/// that achieves exp(i*ArcTan(2)*Z), also known as the "V gate"
/// by Paetznick & Svore.
///
/// # References
/// - [ *Adam Paetznick, Krysta M. Svore*,
/// Quantum Information & Computation 14(15 & 16): 1277-1301 (2014)
/// ](https://arxiv.org/abs/1311.1074)
/// For circuit diagram, see file RUS.png.
///
/// # Input
/// ## inputValue
/// Boolean value for input qubit (true maps to One, false maps to Zero)
/// ## inputBasis
/// Pauli basis in which to prepare input qubit
/// ## limit
/// Integer limit to number of repeats of circuit
///
/// # Remarks
/// The program executes a circuit on a "target" qubit using an "auxiliary"
/// and "resource" qubit. The circuit consists of two parts (red and blue
/// in image).
/// The goal is to measure Zero for both the auxiliary and resource qubit.
/// If this succeeds, the program will have effectively applied an
/// Rz(arctan(2)) gate (also known as V_3 gate) on the target qubit.
/// If this fails, the program reruns the circuit up to <limit> times.
operation AllocateQubitsAndApplyRzArcTan2 (
    inputValue : Bool,
    inputBasis : Pauli,
    measurementBasis : Pauli,
    limit : Int
) : Result[] {
    use (auxiliary, resource, target) = (Qubit(), Qubit(), Qubit());
    
    // Prepare target qubit in a zero- or one-state, based on input value
    if (inputValue) {
        X(target);
    }
    PreparePauliEigenstate(inputBasis, target);
    
    within {
        H(auxiliary);
        H(resource);
    }
    apply {
        ApplyRzArcTan2(limit, auxiliary, resource, target);

        // let rotation angle = 2.0 * ArcTan(2.0);
        let rotationAngle =
        2.2142974355881810060341309203570740801400952908028652933530784148;
        Rz(rotationAngle, target); // Rotate back to initial state
    }
    
    return [M(auxiliary), M(resource), Measure([measurementBasis], [target])];
}

/// # Summary
/// Apply Rz(arctan(2)) on qubits using repeat until success algorithm.
/// Updated to use for-loop sytle.
operation ApplyRzArcTan2 (
    limit : Int,
    auxiliary : Qubit,
    resource : Qubit,
    target : Qubit
) : Unit {
    mutable (result1, result2) = (false, false);
    
    // Run Part 1 of the program.
    for _ in 1..limit {
        if not result1 or not result2 {
            set result1 = ApplyAndMeasurePart1(auxiliary, resource) == Zero;
            
            // We’ll only run Part 2 if Part 1 returns Zero.
            // Otherwise, we’ll skip and rerun Part 1 again.
            if result1 {
                set result2 = ApplyAndMeasurePart2(resource, target) == Zero;
                
                if not result2 {
                    Z(resource); // Reset resource
                    Adjoint Z(target); // Correct effective Z rotation on target
                }
            } else {
                Z(auxiliary);
                Reset(resource);
                H(resource);
            }
        }
    }
}

/// # Summary
/// Apply part 1 of RUS circuit and measure
/// auxiliary qubit in Pauli X basis
operation ApplyAndMeasurePart1(auxiliary : Qubit, resource : Qubit) : Result {
    within {
        T(auxiliary);
    } 
    apply {
        CNOT(resource, auxiliary);
    }
    return Measure([PauliX], [auxiliary]);
}

/// # Summary
/// Apply part 2 of RUS circuit and measure
/// resource qubit in Pauli X basis
operation ApplyAndMeasurePart2(resource : Qubit, target : Qubit) : Result {
    T(target);
    Z(target);
    CNOT(target, resource);
    T(resource);
    return Measure([PauliX], [resource]);
}

## 3. Setup the Experiment
We create three wrapper operations for the code above, each one increasting the number of allowed retries compared to the previous one, so we can see the percentage of resulting success when the repeat logic is allowed to execute.

In [ ]:
# We declare the functions we will define in Q# as a callable so that Python recognizes the symbols,
# then define those functions calling code from the algorithm above.
TestRUSLimit0: any = None
TestRUSLimit1: any = None
TestRUSLimit2: any = None

In [ ]:
%%qsharp

/// # Summary
/// Test the RzArcTan2 algorithm using Pauli-X for preparation and measurement,
/// with no repeat retries.
operation TestRUSLimit0() : Result[] {
    return AllocateQubitsAndApplyRzArcTan2(false, PauliX, PauliX, 0);
}

/// # Summary
/// Test the RzArcTan2 algorithm using Pauli-X for preparation and measurement,
/// using a limit of 1 looping retries.
operation TestRUSLimit1() : Result[] {
    return AllocateQubitsAndApplyRzArcTan2(false, PauliX, PauliX, 1);
}

/// # Summary
/// Test the RzArcTan2 algorithm using Pauli-X for preparation and measurement,
/// using a limit of 2 looping retries.
operation TestRUSLimit2() : Result[] {
    return AllocateQubitsAndApplyRzArcTan2(false, PauliX, PauliX, 2);
}

## 4. Run the Experiment
We can now submit and wait on each of the above experiments, gathering the resulting histograms into repective variables.

In [ ]:
job0 = qsharp.azure.submit(TestRUSLimit0, jobName="RUS Limit 0", shots=100)
while (job0.status not in ["Succeeded", "Failed", "Cancelled"]):
    print(".", end="", flush=True)
    time.sleep(5)
    job0 = qsharp.azure.status()
results0 = qsharp.azure.output()
results0

In [ ]:
job1 = qsharp.azure.submit(TestRUSLimit1, jobName="RUS Limit 1", shots=100)
while (job1.status not in ["Succeeded", "Failed", "Cancelled"]):
    print(".", end="", flush=True)
    time.sleep(5)
    job1 = qsharp.azure.status()
results1 = qsharp.azure.output()
results1

In [ ]:
job2 = qsharp.azure.submit(TestRUSLimit2, jobName="RUS Limit 2", shots=100)
while (job2.status not in ["Succeeded", "Failed", "Cancelled"]):
    print(".", end="", flush=True)
    time.sleep(5)
    job2 = qsharp.azure.status()
results2 = qsharp.azure.output()
results2

## 5. Plot the Results
Finally, we plot the results. When run against the simulator, each successive experiment should show increasing probability of success, indicating that the more allowed runs of RUS further refine the effectiveness of the program. Note that when run with the syntax checker target that returns all zeros, the graph should show a 1.0 result for every repeat limit value.

In [ ]:
from matplotlib import pyplot

success = "[0, 0, 0]"
xticks = [0, 1, 2]
pyplot.title("Probability Success")
pyplot.bar(xticks, [results0[success], results1[success], results2[success]])
pyplot.xticks([0, 1, 2])
pyplot.xlabel("Repeat Limit")
pyplot.ylabel("Probability")


pyplot.show()